<a href="https://colab.research.google.com/github/Emfyr/emfyr.github.io/blob/master/Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Change account because "no gpu backend available" and sometimes the notebook not opening at all.

Also because google throttle.

# This will download files (most likely music) from my mega import folder to mega folder in /content/, extract zip,rar,7z files, delete the archive files, convert all flac files to opus, delete flac files, archive them with each single archive being 200mb in the rar folder of /My Drive/

Don't forget to change name of the archive.

Ond't forget to change directory to /content/mega/ before downloading files form mega.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!apt install megatools
!apt install opus-tools
!apt install imagemagick
!mkdir mega
%cd "/content/drive/My Drive/"
!mkdir rar
%cd "/content/mega/"
!megadl https://mega.nz/#F!mkd2haiZ!ytWf32-HZWBxNWrnoFjXhg
!find -name "*.7z" -exec bash -c '7z -o* x "{}"' {} \;
!find -name "*.rar" -exec bash -c '7z -o* x "{}"' {} \;
!find -name "*.zip" -exec bash -c '7z -o* x "{}"' {} \;
!find . -type f -iname \*.7z -delete
!find . -type f -iname \*.zip -delete
!find . -type f -iname \*.rar -delete
!find -name "*.png" -exec bash -c 'convert "{}" -quality 90 "${0/.png}.jpg"' {} \;
!find -name "*.flac" -exec bash -c 'opusenc --discard-pictures --bitrate 192 "{}" "${0/.flac}.opus"' {} \;
!find . -type f -iname \*.flac -delete
%cd "/content/drive/My Drive/rar/"
!7z -v200m -mx1 -mmt40 a "album_name" "/content/mega/"
%cd "/content/"
!rm -r ./mega/

# This download manga to /My Drive/ this time (because I am afraid they will get lost in /content/), move to /content/Manga/, then archive them into group of 200mb each file (directly saving them into /My Drive/).

Don't forget to change the archive name.

If downloading from mangadex, always use
    
    --chapter-filter "lang == 'en'" 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd "/content/drive/My Drive/"
!pip install gallery-dl
!gallery-dl "Link"
%cd "/content/"
!mkdir Manga
!mv "/content/drive/My Drive/gallery-dl/" "/content/Manga/"
%cd "/content/drive/My Drive/"
!mkdir ./manga/
%cd "/content/drive/My Drive/manga"
!7z -v200m -mx1 -mmt40 a "name_of_manga" "/content/Manga/"
!rm -r "/content/Manga/"
%cd "/content/drive/My Drive/"

# [METHOD: 1] (Inefficient method) (Thanks to FKLC anyway for introducing me to colab) This will download a torrent content to /My Drive/Torrent/ [THANKS TO FKLC]

Use when..... well I would know :)
Have resume support too :)
If a torrent is too big, it may not show progress bar. Stop the execution of the progess showing cell and rerun the cell (the last cell). It will show prgress bar now.
[EDIT: I was retarded. I have to wait for TORRENT folder to show up before executing the last cell.]

If lossless music, thenexecute the first section one you know (Modify a little).

GPU acceleration for 384GB space.

If torrent contents are huge, then MOVE some contents to /My Drive/, open gdrive, get sharable link, import the incomplete contents to other gdrive, open new notebook, 7z the shit out of that, delete folder from the 1st gdrive and repeat the process.

In [0]:
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

In [0]:
from google.colab import drive

drive.mount("/content/drive")

In [0]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

In [0]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


In [0]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


# [METHOD: 2] (My own method) Download to /content/drive/My Drive/Torrent


-This is important. If link is a magnet link, always use

    -d "output directory"
    
-Use 

    -T file.torrent

if the file is?? Can you guess what?? A torrrent file. Bet you were surprised.


-Following only work if file is torrent. (This show files in torrent. Use to get number which can then be put in --select-file argument.)

    -S


-This would well slect files to download

    --select-file=x-y,z,a-b 






..

..

..

..

..

..

..

..

..

..


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!apt install aria2

In [0]:
!aria2c --max-connection-per-server=16 --split=16 --seed-time=0 --download-result=full --auto-save-interval=20 --bt-max-open-files=10000 --bt-max-peers=0 --save-session-interval=20 "magnet link"

#Use this after downloading "audio torrents", to convert to opus, and then archive.

In [0]:
!apt install megatools
!apt install opus-tools
!apt install imagemagick

!find -name "*.7z" -exec bash -c '7z -o* x "{}"' {} \;
!find -name "*.rar" -exec bash -c '7z -o* x "{}"' {} \;
!find -name "*.zip" -exec bash -c '7z -o* x "{}"' {} \;
!find . -type f -iname \*.7z -delete
!find . -type f -iname \*.zip -delete
!find . -type f -iname \*.rar -delete

!find -name "*.png" -exec bash -c 'convert "{}" -quality 90 "${0/.png}.jpg"' {} \;
!find -name "*.flac" -exec bash -c 'opusenc --discard-pictures --bitrate 192 "{}" "${0/.flac}.opus"' {} \;
!find . -type f -iname \*.flac -delete

In [0]:
%cd "/content/drive/My Drive/rar/"
!7z -v200m -mx1 -mmt40 a "album_name" "/content/mega/"
%cd "/content/"